In [1]:
import lib.datautility as du
import numpy as np
import lib.tf_network2 as tfnet
import lib.evaluationutility as eu
from lib.tf_network2 import Network, Cost, Normalization, Optimizer
import tensorflow as tf
import session_info

In [2]:
import zipfile
with zipfile.ZipFile('resources/resources.zip','r') as zf:
    zf.extractall('resources/')

In [3]:
data, headers = du.read_csv('resources/remnant.csv')
du.print_descriptives(data, headers)

data, headers = du.read_csv('resources/experimental.csv')
du.print_descriptives(data, headers)

-- loading resources/remnant.csv...(35.02%) 

KeyboardInterrupt: 

In [ ]:
key = [1,2,0]
iden = [4]
cov = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
lab = [[3]]
order = 7

seq = tfnet.format_sequence_from_file('resources/remnant.csv',
                                          key,lab,cov,iden,order)
print('formatting feature columns...')
for i in range(len(seq['x'])):
    for j in range(len(seq['x'][i])):
        try:
            seq['x'][i][j] = np.array(seq['x'][i][j], dtype=np.float32)
        except TypeError:
            print('error...')
            print(seq['x'][i])
            exit(1)
seq['x'] = tfnet.sequence_impute_missing(seq['x'])

print('formatting output labels...')
seq['y'] = tfnet.use_last_multi_label(seq['y'], 0)

print('done!')

In [ ]:
key = [1,2,0]
iden = [4]
cov = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
lab = [[3]]
order = 7

seqex = tfnet.format_sequence_from_file('resources/experimental.csv',
                                          key,lab,cov,iden,order)
print('formatting feature columns...')
for i in range(len(seqex['x'])):
    for j in range(len(seqex['x'][i])):
        try:
            seqex['x'][i][j] = np.array(seqex['x'][i][j], dtype=np.float32)
        except TypeError:
            print('error...')
            print(seqex['x'][i])
            exit(1)
seqex['x'] = tfnet.sequence_impute_missing(seqex['x'])

print('formatting output labels...')
seqex['y'] = tfnet.use_last_multi_label(seqex['y'], 0)

print('done!')

In [ ]:
n_cov = len(seq['x'][0][0])

max_epochs = 200
use_validation = True
hidden = [50]
batch = [64]
layers = [1]
keep = [.5]
step = [1e-3]
perf = []
threshold = [0.0001]
optimizer = [Optimizer.ADAM]

tf.compat.v1.disable_eager_execution()

tfnet.describe_multi_label(seq['y'], True)

In [ ]:
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(1)
np.random.seed(1)

net = Network('study2_model').add_input_layer(n_cov, normalization=Normalization.Z_SCORE)
net.add_lstm_layer(hidden[0], peepholes=True)

net.begin_multi_output(cost_methods=[Cost.BINARY_CROSS_ENTROPY])
net.add_dropout_layer(1, keep=keep[0], activation=tf.nn.sigmoid)
net.end_multi_output()

net.set_default_cost_method(Cost.CROSS_ENTROPY)
net.set_optimizer(optimizer[0])

net.train(x=seq['x'],
          y=seq['y'],
          step=step[0],
          use_validation=True,
          max_epochs=max_epochs, threshold=threshold[0], batch=batch[0])

In [ ]:
net.save_model_weights()      # comment to use a previously-saved model
#
# net.build()                 # uncomment to use previously-saved model
# net.restore_model_weights() # uncomment to use previously-saved model

In [ ]:
print('writing predictions to file...')
du.write_csv(tfnet.flatten_sequence(np.array([np.array([t[-1]]) for t in net.predict(seqex['x'])[0]]),
                                        seqex['key'].reshape((-1,3))), 'model_predictions.csv',
             ['user_id','target_assignment_id','target_sequence_id','pcomplete'])
print('done!')

In [ ]:
import graph_def_editor 
import warnings
import time
import csv
import sys
import os
import pandas 
from sklearn.metrics import f1_score, r2_score
from sklearn.metrics import mean_squared_error
from skll.metrics import kappa as kpa
import pickle
import psycopg2 as pg
import functools
import Levenshtein as lev
from nltk.tokenize import word_tokenize

session_info.show()

In [ ]:
net.session.close()